In [7]:
# 3D Event Display for events in the ANNIE Tank

%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas
from mpl_toolkits.mplot3d import Axes3D

# For some reason specifying the backend for matplotlib never works. If running on a MAC just run this cell twice

# ----------------------------------------------------
# Parameterization 

# path
position = 'gamma_swarm'
folder = 'Extracted_Data/'
path_charge = folder + 'charge_event_' + position + '.dat'

primary_particle = r'$\gamma$'                        # MC particle (write as equation, pick from list:   r'$e^-$',  r'$E_{\gamma}$')

How_many_events = 3                                   # How many events should be displayed/saved?  
                                                      # (setting it equal to 'all' will run over all events)
                                                      # for the 3D display, don't run too many events, as they are all in interactive windows

event_offset = 3                                      # Adjust to plot a later event (this is the starting event number you wish to plot)
                                                      # (if you wish to run over all events, set this = 0)

Name = '3D_Plots/MC Deexcitation '                    # Name of the Plot/Event?

plot_other_PMTs = False                               # plot the non-hit PMTs?
                                                      # on = good for visualizing the tank geometry
                                                      # off = good for visualizing the event and the PMTs that were hit

PMT_markersize = 100                                  # default is 100


# ----------------------------------------------------
### First, Load in the Detector Geometry ###

# Read Geometry.csv file to get PMT location info
df = pandas.read_csv('FullTankPMTGeometry.csv')

channel_number = []; location = []; panel_number = []
x_position = []; y_position = []; z_position = []

# The LoadGeometry File does not share the same origin point as the WCSim output after ToolAnalysis.
# Need to adjust to ensure the (0,0,0) point (center of the tank) is the same.
# This offset is confirmed by plotting it without adjustments (see below, commented)

# vertical center (y) is at a height of y = -14.46 cm
# x-axis is fine
# z-axis (beamline) is offset by 1.681 m
# tank center is at (0,-14.46, 168.1) [cm]

for i in range(len(df['channel_num'])):   # loop over PMTs
    channel_number.append(df['channel_num'][i])
    location.append(df['detector_tank_location'][i])
    x_position.append(df['x_pos'][i]+0)
    y_position.append(df['y_pos'][i]+0.1446)
    z_position.append(df['z_pos'][i]-1.681)
    panel_number.append(df['panel_number'][i])

print('\nPMT Geometry Loaded')


#################################################################################
# For each clustered event (just referred to as cluster from now on), load in the 
# event-level information (like charge balance, number of hits, etc..)

event_header = np.loadtxt(path_charge, dtype = str, delimiter = None, max_rows = 1)
event_data = np.loadtxt(path_charge, dtype = float, delimiter = None, skiprows = 1)

clustereventNumber = event_data.T[0]
clusterCharge = event_data.T[1]
clusterPE = event_data.T[2]
clusterMaxPE = event_data.T[3]
clusterChargeBalance = event_data.T[4]
clusterHits = event_data.T[5]

N_events = len(clustereventNumber)

# Now load in the hits-level information
path_hits = folder + 'cluster_hits_' + position + '.dat'
hits_header = np.loadtxt(path_hits, dtype = str, delimiter = None, max_rows = 1)
hits_data = np.loadtxt(path_hits, dtype = float, delimiter = None, skiprows = 1)

Channel = [[] for i in range(N_events)]
hitT = [[] for i in range(N_events)]      # The x,y,z is adjusted correctly in the ToolChain for this Event Display
hitX = [[] for i in range(N_events)]
hitY = [[] for i in range(N_events)]
hitZ = [[] for i in range(N_events)]
hitQ = [[] for i in range(N_events)]
hitPE = [[] for i in range(N_events)]

count = 0
for j in range(len(hits_data.T[0])):    # loop over all hits (N events x M hits per event)
    if (j == 0):
        Channel[count].append(hits_data.T[1][j])
        hitT[count].append(hits_data.T[2][j])      # the hitX, hitY, hitZ contain the position of the
        hitX[count].append(hits_data.T[3][j])      # PMTs that were hit
        hitY[count].append(hits_data.T[4][j])      
        hitZ[count].append(hits_data.T[5][j])
        hitQ[count].append(hits_data.T[6][j])
        hitPE[count].append(hits_data.T[7][j])
        
    elif (j != 0):
        if hits_data.T[0][j] == hits_data.T[0][j-1]:
            Channel[count].append(hits_data.T[1][j])
            hitT[count].append(hits_data.T[2][j])
            hitX[count].append(hits_data.T[3][j])
            hitY[count].append(hits_data.T[4][j])
            hitZ[count].append(hits_data.T[5][j])
            hitQ[count].append(hits_data.T[6][j])
            hitPE[count].append(hits_data.T[7][j])
        else:
            count = count + 1
            Channel[count].append(hits_data.T[1][j])
            hitT[count].append(hits_data.T[2][j])
            hitX[count].append(hits_data.T[3][j])
            hitY[count].append(hits_data.T[4][j])
            hitZ[count].append(hits_data.T[5][j])
            hitQ[count].append(hits_data.T[6][j])
            hitPE[count].append(hits_data.T[7][j])
            
# Load in the MC Truth information (there will be more events than clusters -- need to sort and assign them)
path_truth = folder + 'truth_' + position + '.dat'
truth_header = np.loadtxt(path_truth, dtype = str, delimiter = None, max_rows = 1)
truth_data = np.loadtxt(path_truth, dtype = float, delimiter = None, skiprows = 1)

eventNumber = truth_data.T[0]  # Event Number
vtX = truth_data.T[1]          # {vertex information   
vtY = truth_data.T[2]          # (units of cm)
vtZ = truth_data.T[3]          # }
t0 = truth_data.T[4]           # "vertex time" i.e. initial time
dirX = truth_data.T[5]         # {direction vectors of primary particle
dirY = truth_data.T[6]         # ...
dirZ = truth_data.T[7]         # }
Energy = truth_data.T[8]       # initial energy of the primary particle
Track_Length = truth_data.T[9] # track length of the primary particle in water (distance from start point to stop point or the
                               # distance from the start vertex to a tank wall (if the particle exited))  (units of meters)

# sort events that dont have an associated cluster event number
vtX = [vtX[int(x)]/100 for x in eventNumber if x in clustereventNumber]
vtY = [vtY[int(x)]/100 for x in eventNumber if x in clustereventNumber]
vtZ = [vtZ[int(x)]/100 for x in eventNumber if x in clustereventNumber]
# divide the verticies by 100 to convert to meters
t0 = [t0[int(x)] for x in eventNumber if x in clustereventNumber]
dirX = [dirX[int(x)] for x in eventNumber if x in clustereventNumber]
dirY = [dirY[int(x)] for x in eventNumber if x in clustereventNumber]
dirZ = [dirZ[int(x)] for x in eventNumber if x in clustereventNumber]
Energy = [Energy[int(x)] for x in eventNumber if x in clustereventNumber]
Track_Length = [Track_Length[int(x)] for x in eventNumber if x in clustereventNumber]

#################################################################################
# We can also create some custom arrays

# Charge and hits
Sum_PE = [[] for i in range(N_events)]       # summed P.E. on each PMT
hits_per_PMT = [[] for i in range(N_events)] # number of hits on each PMT
unique_PMTs = [[] for i in range(N_events)]  # unique hit PMTs
for i in range(N_events):
    u, c = np.unique(Channel[i], return_counts=True)
    unique_PMTs[i].append(u.tolist())        # prevents this list from becoming a numpy array
    hits_per_PMT[i].append(c.tolist())       # (only included because we call the method index() later on, which only works on lists)
for i in range(N_events):
    for j in range(len(unique_PMTs[i][0])):
        pe = 0.
        for k in range(len(Channel[i])):
            if unique_PMTs[i][0][j] == Channel[i][k]:
                pe = pe + hitPE[i][k]
        Sum_PE[i].append(pe)
        
# There exists a small descrepancy between clusterMaxPE and the summed Max PE per PMT
# in the former, the maxPE is defined as the maximum PE of a given hit. The latter accounts
# for multiple hits on the same PMT, so it is the maximum summed charge over all PMTs. The
# difference over the entirity of the events is small though. We will use the Sum_PE for our event display.
        
# # # # # # # # # # # # # # # # # # # # # # # # # # #
# True Vertex and direction information
origin = np.zeros([N_events,3])
dir_vector = np.zeros([N_events,3])
for i in range(N_events):
    origin[i][0] = vtZ[i]; dir_vector[i][0] = dirZ[i]
    origin[i][1] = vtX[i]; dir_vector[i][1] = dirX[i]
    origin[i][2] = vtY[i]; dir_vector[i][2] = dirY[i]
    
# Track Length can be a proxy for the magnitude of the direction vector

# # # # # # # # # # # # # # # # # # # # # # # # # # #
# We can also define whether an event triggered an extended readout:
# (events with max p.e. > 7 for a given PMT)
ext_readout_trig = []
for i in range(N_events):
    if clusterMaxPE[i] > 7.:
        ext_readout_trig.append(True)
    else:
        ext_readout_trig.append(False)
        
# # # # # # # # # # # # # # # # # # # # # # # # # # #

print('\nData Loaded')
print('\n#################')
print('\nN Events = ', N_events)

# # # # # # # # # # # # # # # # # # # # # # # # # # #

print('\n###########################')

if How_many_events == 'all':              # specify in beginning; if running over all events
    How_many_events = N_events


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
############################ Charge Plot ##############################
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

for i in range(event_offset, How_many_events + event_offset):

    print('\nEvent ' + str(i))

    fig = plt.figure(figsize = (15,10))
    
    ax = fig.add_subplot(projection='3d', computed_zorder=False)   # zorder was not manual as of 3.5.0
    
    fig.patch.set_facecolor('xkcd:black')

    # Plot PMTs that were not hit  (assigned black)
    if plot_other_PMTs == True:
        for cn in range(len(channel_number)):
            if channel_number[cn] not in unique_PMTs[i][0]:   # construct base geometry of PMTs not hit
                ax.scatter(z_position[cn], x_position[cn], y_position[cn], s = PMT_markersize, color = 'black', zorder = 5)

    for hit in range(len(Channel[i])):     # loop over hit PMTs
        index = unique_PMTs[i][0].index(Channel[i][hit])   # Use summed charge per PMT
        p = ax.scatter(hitZ[i][hit], hitX[i][hit], hitY[i][hit], c = Sum_PE[i][index], cmap=plt.cm.plasma,
                    s = PMT_markersize, vmin = 0, vmax = max(Sum_PE[i]), alpha = 1, zorder = 5)

    cb = plt.colorbar(p)   # colorbar depecting the amount of charge recorded on each PMT
    cb.set_label(label = 'Photoelectrons', color = 'white')
    cb.ax.yaxis.set_tick_params(color='white')
    cb.outline.set_edgecolor('white')
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='white')

    # # # # # # # # # # # # # # # # #

    # Statistics
    hits_text = str(int(clusterHits[i])) + ' hits / ' + str(round(clusterPE[i],2)) + ' p.e.'
    ax.text2D(0.8,0.95,hits_text,size = 12,transform = ax.transAxes, color = 'white')
    PMT_text = 'PMTs with Hits: '+ str(len(unique_PMTs[i][0]))
    ax.text2D(0.8,0.90,PMT_text,size = 12,transform = ax.transAxes, color = 'white')
    chargebalance = r'$Q_{b}$' + ' = ' + str(round(clusterChargeBalance[i],3)) + ' ; ' + r'$PE_{max}$' + ' = ' + str(round(max(Sum_PE[i]),2))
    ax.text2D(0.77,0.85,chargebalance,size = 12,transform = ax.transAxes, color = 'white')
    
    # # # # # # # # # # # # # # # # #
    
    # Plot vertex truth position - figure is (z,x,y) but loaded origin is (x,y,z)
    ax.scatter(origin[i][0],origin[i][1],origin[i][2], s = PMT_markersize, color = 'red', marker = '*', zorder = 1, label = 'Truth Vertex')
    
    # Plot the truth direction (momentum)
    scale = Track_Length[i]   # Scale of arrow will be the real track length of the primary gamma
    ax.quiver(origin[i][0], origin[i][1], origin[i][2], dirZ[i]*scale, dirX[i]*scale, dirY[i]*scale, 
            color = 'red', label = 'Primary ' + primary_particle + ' initial momentum and final track length', zorder = 1)
    
    # Important to note that the track length is simply the difference between the start and stop vertex of the primary particle. 
    # The initial direction is truth, but the actual direction as the primary particle propagates may change. 
    # Therefore, the quiver within the plot is more of a visual representation of the propagation length of the initial particle.

    ax.set_aspect('auto')
    ax.set_xlabel('z [m]', color = 'white')
    ax.set_ylabel('x [m]', color = 'white')
    ax.set_zlabel('y [m]', color = 'white')
    ax.xaxis.set_tick_params(color='white')
    ax.yaxis.set_tick_params(color='white')
    ax.zaxis.set_tick_params(color='white')
    plt.setp(plt.getp(ax.axes, 'xticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'yticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'zticklabels'), color='white')

    ax.set_facecolor('xkcd:black')

    # Create cubic bounding box to simulate equal aspect ratio
    max_range = np.array([max(z_position)-min(z_position), max(x_position)-min(x_position), max(y_position)-min(y_position)]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(max(z_position)+min(z_position))
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(max(x_position)+min(x_position))
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(max(y_position)+min(y_position))
    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')

    plt.title('ANNIE Tank 3D Event ' + str(i) + ' | Charge Information', color = 'white')
    plt.legend(loc = 'upper left')
    
    path = Name + ' Event ' + str(i) +  ' _ Charge Plot.png'
    plt.savefig(path,dpi=300, bbox_inches='tight', pad_inches=.3,facecolor = 'black')
    plt.show()


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
############################ Timing Plot ##############################
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    
    fig = plt.figure(figsize = (15,10))
    
    ax = fig.add_subplot(projection='3d', computed_zorder=False)   # zorder was not manual as of 3.5.0
    
    fig.patch.set_facecolor('xkcd:black')

    # Plot PMTs that were not hit  (assigned black)
    if plot_other_PMTs == True:
        for cn in range(len(channel_number)):
            if channel_number[cn] not in unique_PMTs[i][0]:   # construct base geometry of PMTs not hit
                ax.scatter(z_position[cn], x_position[cn], y_position[cn], s = PMT_markersize, color = 'black', zorder = 5)

    # We want to assign colors depicting the timing of each PMT hit
    # For now, there will be degeneracy where the same PMT is hit multiple times (the scatterpoints will overlay)
    # Could change this in the future to an average hit time - for now just keep the final hit time

    p = ax.scatter(hitZ[i], hitX[i], hitY[i], c = hitT[i], cmap=plt.cm.cool, vmin = t0[i], vmax = max(hitT[i]), 
                   s = PMT_markersize, alpha = 1, zorder = 5)

    cb = plt.colorbar(p)   # colorbar depecting the amount of charge recorded on each PMT
    cb.set_label(label = 'Time [nanoseconds]', color = 'white')
    cb.ax.yaxis.set_tick_params(color='white')
    cb.outline.set_edgecolor('white')
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='white')
    
    # # # # # # # # # # # # # # # # #

    # Statistics
    hits_text = str(int(clusterHits[i])) + ' hits / ' + str(round(clusterPE[i],2)) + ' p.e.'
    ax.text2D(0.8,0.95,hits_text,size = 12,transform = ax.transAxes, color = 'white')
    PMT_text = 'PMTs with Hits: '+ str(len(unique_PMTs[i][0]))
    ax.text2D(0.8,0.90,PMT_text,size = 12,transform = ax.transAxes, color = 'white')
    
    import statistics as st
    t_text = r'$(t_{0},t_{first},t_{median})$' + ' = (' + str(int(t0[i])) + ',' +  str(int(min(hitT[i]))) + ',' + str(int(st.median(hitT[i]))) + ') ns'
    ax.text2D(0.75,0.85,t_text,size = 12,transform = ax.transAxes, color = 'white')
    
    # # # # # # # # # # # # # # # # #

    # Plot vertex truth position - figure is (z,x,y) but loaded origin is (x,y,z)
    ax.scatter(origin[i][0],origin[i][1],origin[i][2], s = PMT_markersize, color = 'red', marker = '*', zorder = 1, label = 'Truth Vertex')
    
    # Plot the truth direction (momentum)
    scale = Track_Length[i]   # Scale of arrow will be the real track length of the primary gamma
    ax.quiver(origin[i][0], origin[i][1], origin[i][2], dirZ[i]*scale, dirX[i]*scale, dirY[i]*scale, 
            color = 'red', label = 'Primary ' + primary_particle + ' initial momentum and final track length', zorder = 1)

    ax.set_aspect('auto')
    ax.set_xlabel('z [m]', color = 'white')
    ax.set_ylabel('x [m]', color = 'white')
    ax.set_zlabel('y [m]', color = 'white')
    ax.xaxis.set_tick_params(color='white')
    ax.yaxis.set_tick_params(color='white')
    ax.zaxis.set_tick_params(color='white')
    plt.setp(plt.getp(ax.axes, 'xticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'yticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'zticklabels'), color='white')

    ax.set_facecolor('xkcd:black')

    # Create cubic bounding box to simulate equal aspect ratio
    max_range = np.array([max(z_position)-min(z_position), max(x_position)-min(x_position), max(y_position)-min(y_position)]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(max(z_position)+min(z_position))
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(max(x_position)+min(x_position))
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(max(y_position)+min(y_position))
    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')

    plt.title('ANNIE Tank 3D Event ' + str(i) + ' | Hit Time Information', color = 'white')
    plt.legend(shadow=True, loc = 'upper left')

    path = Name + ' Event ' + str(i) + ' _ Timing Plot.png'
    plt.savefig(path,dpi=300, bbox_inches='tight', pad_inches=.3,facecolor = 'black')

    plt.show()


print('\ndone')  

Using matplotlib backend: MacOSX

PMT Geometry Loaded

Data Loaded

#################

N Events =  874

###########################

Event 3

Event 4

Event 5

done
